# Instalação do pydantic

In [1]:
pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.6 MB/s eta 0:00:00


# Importações e setups necessários

In [2]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

# Regras para a criação da senha e do nome

In [3]:
# Obriga a senha a ter ao menos 8 caracteres, uma maiúscula, uma minúscula e um número
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
# Obriga o nome a ter apenas letras e ao menos 2 caracteres
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

# Criação das classes Role e User

In [9]:
''' Isso cria um enum chamado Role para cada cargo, dessa forma cada cargo recebe um
valor inteiro, ajudando a organizar e restringir valores da forma desejada'''
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


'''Isso cria um modelo de usuário, especificando que este pode ter nome, email,
senha e cargo, o Field é usado para adicionar metadados a cada atributo, como exemplos
ou regras'''
class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )
# Adição de métodos de validação, o field_validator valida o nome usanod o VALID_NAME_REGEX
    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v
# O validate_role valida o cargo garantindo que é um enum válido independentemente da
# forma que ele for informado(str, int, enum)
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

# validate_user_pre realiza uma validação em todos os dados do usuário antes de
# criar o objeto, checa se senha e nome estão presentes, se a senha não é igual
# ao nome, se a senha se encaixa no VALID_PASSWORD_REGEX, e usa uma HASH na senha
# para garantir a segurança
    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

# validate_user_post realiza uma validação após a criação do objeto, define que
# apenas o user Arjan pode ter o cargo de admin
    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

# Serializações
# controle de como o user é convertido a outro tipo de dado
# serialize_role garante que o cargo seja salvo com seu nome
    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

# serialize_number garante que o objeto User seja serializado incluindo apenas
# o cargo e o nome ou as informações desejadas da forma configurada
    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

# Definição da função main e execução

In [10]:
'''Criação da função main, criação de um dicionário chamado data que conterá
as informações do usuário, esses dados serão usados para criar o Objeto user.
Em seguida valida e cria o objeto, confere se foi criado corretamente e o
serializa em diferentes formatos.'''

def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")


if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}
